<a href="https://colab.research.google.com/github/danadler-dev/voice_leading/blob/main/VoiceLeading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd
from enum import Enum

class Scale(Enum):
    MAJOR = 1
    H_MINOR = 2
    M_MINOR = 3

############################################################
# Class to generate 3,4,5 part diatonic chords for any scale
# For example: 4 part chords in F Major:
# c = Diatonic("F", Scale.MAJOR)
# c.chords(4)
############################################################

class Diatonic:
  _tonic = '?'
  _scale = []
  _notes_flat  = ["C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab", "A", "Bb", "B"]
  _Notes_flat  = pd.Series(_notes_flat)

  _notes_sharp = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
  _Notes_sharp = pd.Series(_notes_sharp)

  def _mk_scale(self, intervals):
    sharp = lambda t: ('#' in t) or (t in ["D", "E", "G", "A", "B"])
    notes = self._notes_sharp if sharp(self._tonic) else self._notes_flat
    Notes = self._Notes_sharp if sharp(self._tonic) else self._Notes_flat
    offset = notes.index(self._tonic)
    ivals = [(x + offset) % 12 for x in intervals]
    return list(Notes[ivals])

  def _major(self):
    return self._mk_scale([0,2,4,5,7,9,11])

  def _mel_minor(self):
    return self._mk_scale([0,2,3,5,7,9,11])

  def _har_minor(self):
    return self._mk_scale([0,2,3,5,7,8,11])

  def __init__(self, tonic:str, kind:Scale ):
    self._tonic = tonic
    if kind == Scale.MAJOR:
      self._scale = self._major()
    elif kind == Scale.M_MINOR:
      self._scale = self._mel_minor()
    else:
      self._scale = self._har_minor() 

  def chords(self, num=3):
    next = lambda x, n=1: self._scale[(self._scale.index(x)+n) % len(self._scale)]
    return [tuple([next(c,2*i) for i in range(num)]) for c in self._scale]



In [100]:
# 4-part diatonic chords in F melodic minor
c = Diatonic("F", Scale.M_MINOR)
assert c.chords(4) == [
 ('F', 'Ab', 'C', 'E'),
 ('G', 'Bb', 'D', 'F'),
 ('Ab', 'C', 'E', 'G'),
 ('Bb', 'D', 'F', 'Ab'),
 ('C', 'E', 'G', 'Bb'),
 ('D', 'F', 'Ab', 'C'),
 ('E', 'G', 'Bb', 'D')]

In [101]:
# 5-part diatonic chords in C major
c = Diatonic("C", Scale.MAJOR)
assert c.chords(5) == [
 ('C', 'E', 'G', 'B', 'D'),
 ('D', 'F', 'A', 'C', 'E'),
 ('E', 'G', 'B', 'D', 'F'),
 ('F', 'A', 'C', 'E', 'G'),
 ('G', 'B', 'D', 'F', 'A'),
 ('A', 'C', 'E', 'G', 'B'),
 ('B', 'D', 'F', 'A', 'C') 
]

In [102]:
# 5-part diatonic chords in D major
d = Diatonic("D", Scale.MAJOR)
assert d.chords(4) == [
 ('D', 'F#', 'A', 'C#'),
 ('E', 'G', 'B', 'D'),
 ('F#', 'A', 'C#', 'E'),
 ('G', 'B', 'D', 'F#'),
 ('A', 'C#', 'E', 'G'),
 ('B', 'D', 'F#', 'A'),
 ('C#', 'E', 'G', 'B')
]

In [97]:
class VoicingCycle:
  # The 6 cycles as defined in the book intro
  _pc = (5,3,1) # cycle: 5->3->1->5...
  
  def _perm(self, *args):
    return tuple([self._pc[(self._pc.index(a)+1) % len(self._pc)] for a in args[0]])
  
  def __init__(self, chords, pc):
    self.Cycle2 = []; self.Cycle3 = []; self.Cycle4 = []; self.Cycle5 = []; self.Cycle6 = []; self.Cycle7 = []
    self.Cycles = [self.Cycle2, self.Cycle3, self.Cycle4, self.Cycle5, self.Cycle6, self.Cycle7 ]
    for i in range(len(self.Cycles)):
      j=0
      for k in range(len(chords)):
        self.Cycles[i].append(chords[j])
        j = (j + i + 1) % len(chords)
    self._pc = pc  

  # invert a chord like (C,E, G) by a permutation like (3,1,5)
  @classmethod
  def _invert(cls, chord, prm):
    _chord_tone = {1:0, 2:1, 3:1, 4:2, 5:2, 7:3, 9:4}
    loc = tuple([_chord_tone[t] for t in prm]) 
    return tuple([chord[loc[i]] for i in range(len(prm))])
    
  def gen(self, cycle, start):
    for p in start:
      for i in range(len(p)):
        for c in cycle:
          print(self._invert(c, p))
          p = self._perm(p)
        print('---------------')
      print('================')


In [72]:
assert VoicingCycle._invert(('C','E','G'), (3,5,1)) == ('E', 'G', 'C')

In [73]:
# start with C Major Triads
c = Diatonic("C", Scale.MAJOR)

In [104]:
vc = VoicingCycle(c.chords(3), (5,3,1)) # cycle: 5->3->1->5...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle2, voicings )

('C', 'E', 'G')
('A', 'D', 'F')
('G', 'B', 'E')
('F', 'A', 'C')
('D', 'G', 'B')
('C', 'E', 'A')
('B', 'D', 'F')
---------------
('G', 'C', 'E')
('F', 'A', 'D')
('E', 'G', 'B')
('C', 'F', 'A')
('B', 'D', 'G')
('A', 'C', 'E')
('F', 'B', 'D')
---------------
('E', 'G', 'C')
('D', 'F', 'A')
('B', 'E', 'G')
('A', 'C', 'F')
('G', 'B', 'D')
('E', 'A', 'C')
('D', 'F', 'B')
---------------
('C', 'G', 'E')
('A', 'F', 'D')
('G', 'E', 'B')
('F', 'C', 'A')
('D', 'B', 'G')
('C', 'A', 'E')
('B', 'F', 'D')
---------------
('G', 'E', 'C')
('F', 'D', 'A')
('E', 'B', 'G')
('C', 'A', 'F')
('B', 'G', 'D')
('A', 'E', 'C')
('F', 'D', 'B')
---------------
('E', 'C', 'G')
('D', 'A', 'F')
('B', 'G', 'E')
('A', 'F', 'C')
('G', 'D', 'B')
('E', 'C', 'A')
('D', 'B', 'F')
---------------


In [105]:
vc = VoicingCycle(c.chords(3), (5,3,1)) # cycle: 5->3->1->5...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle4, voicings) # close voicing, spread voicing

('C', 'E', 'G')
('C', 'F', 'A')
('D', 'F', 'B')
('E', 'G', 'B')
('E', 'A', 'C')
('F', 'A', 'D')
('G', 'B', 'D')
---------------
('G', 'C', 'E')
('A', 'C', 'F')
('B', 'D', 'F')
('B', 'E', 'G')
('C', 'E', 'A')
('D', 'F', 'A')
('D', 'G', 'B')
---------------
('E', 'G', 'C')
('F', 'A', 'C')
('F', 'B', 'D')
('G', 'B', 'E')
('A', 'C', 'E')
('A', 'D', 'F')
('B', 'D', 'G')
---------------
('C', 'G', 'E')
('C', 'A', 'F')
('D', 'B', 'F')
('E', 'B', 'G')
('E', 'C', 'A')
('F', 'D', 'A')
('G', 'D', 'B')
---------------
('G', 'E', 'C')
('A', 'F', 'C')
('B', 'F', 'D')
('B', 'G', 'E')
('C', 'A', 'E')
('D', 'A', 'F')
('D', 'B', 'G')
---------------
('E', 'C', 'G')
('F', 'C', 'A')
('F', 'D', 'B')
('G', 'E', 'B')
('A', 'E', 'C')
('A', 'F', 'D')
('B', 'G', 'D')
---------------


In [106]:
vc = VoicingCycle(c.chords(3), (5,1,3)) # cycle: 5->3->1->5...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle6, voicings) # close voicing, spread voicing

('C', 'E', 'G')
('C', 'E', 'A')
('C', 'F', 'A')
('D', 'F', 'A')
('D', 'F', 'B')
('D', 'G', 'B')
('E', 'G', 'B')
---------------
('E', 'G', 'C')
('E', 'A', 'C')
('F', 'A', 'C')
('F', 'A', 'D')
('F', 'B', 'D')
('G', 'B', 'D')
('G', 'B', 'E')
---------------
('G', 'C', 'E')
('A', 'C', 'E')
('A', 'C', 'F')
('A', 'D', 'F')
('B', 'D', 'F')
('B', 'D', 'G')
('B', 'E', 'G')
---------------
('C', 'G', 'E')
('C', 'A', 'E')
('C', 'A', 'F')
('D', 'A', 'F')
('D', 'B', 'F')
('D', 'B', 'G')
('E', 'B', 'G')
---------------
('E', 'C', 'G')
('E', 'C', 'A')
('F', 'C', 'A')
('F', 'D', 'A')
('F', 'D', 'B')
('G', 'D', 'B')
('G', 'E', 'B')
---------------
('G', 'E', 'C')
('A', 'E', 'C')
('A', 'F', 'C')
('A', 'F', 'D')
('B', 'F', 'D')
('B', 'G', 'D')
('B', 'G', 'E')
---------------


In [107]:
vc = VoicingCycle(c.chords(3), (5,1,3)) # cycle: 5->3->1->5...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle7, voicings) # close voicing, spread voicing

('C', 'E', 'G')
('D', 'F', 'B')
('E', 'A', 'C')
('G', 'B', 'D')
('A', 'C', 'F')
('B', 'E', 'G')
('D', 'F', 'A')
---------------
('E', 'G', 'C')
('F', 'B', 'D')
('A', 'C', 'E')
('B', 'D', 'G')
('C', 'F', 'A')
('E', 'G', 'B')
('F', 'A', 'D')
---------------
('G', 'C', 'E')
('B', 'D', 'F')
('C', 'E', 'A')
('D', 'G', 'B')
('F', 'A', 'C')
('G', 'B', 'E')
('A', 'D', 'F')
---------------
('C', 'G', 'E')
('D', 'B', 'F')
('E', 'C', 'A')
('G', 'D', 'B')
('A', 'F', 'C')
('B', 'G', 'E')
('D', 'A', 'F')
---------------
('E', 'C', 'G')
('F', 'D', 'B')
('A', 'E', 'C')
('B', 'G', 'D')
('C', 'A', 'F')
('E', 'B', 'G')
('F', 'D', 'A')
---------------
('G', 'E', 'C')
('B', 'F', 'D')
('C', 'A', 'E')
('D', 'B', 'G')
('F', 'C', 'A')
('G', 'E', 'B')
('A', 'F', 'D')
---------------


In [108]:
vc = VoicingCycle(c.chords(3), (1,3, 5)) # cycle: 1->3->5->1...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle5, voicings) # close voicing, spread voicing

('C', 'E', 'G')
('B', 'D', 'G')
('A', 'D', 'F')
('A', 'C', 'E')
('G', 'B', 'E')
('F', 'B', 'D')
('F', 'A', 'C')
---------------
('E', 'G', 'C')
('D', 'G', 'B')
('D', 'F', 'A')
('C', 'E', 'A')
('B', 'E', 'G')
('B', 'D', 'F')
('A', 'C', 'F')
---------------
('G', 'C', 'E')
('G', 'B', 'D')
('F', 'A', 'D')
('E', 'A', 'C')
('E', 'G', 'B')
('D', 'F', 'B')
('C', 'F', 'A')
---------------
('C', 'G', 'E')
('B', 'G', 'D')
('A', 'F', 'D')
('A', 'E', 'C')
('G', 'E', 'B')
('F', 'D', 'B')
('F', 'C', 'A')
---------------
('E', 'C', 'G')
('D', 'B', 'G')
('D', 'A', 'F')
('C', 'A', 'E')
('B', 'G', 'E')
('B', 'F', 'D')
('A', 'F', 'C')
---------------
('G', 'E', 'C')
('G', 'D', 'B')
('F', 'D', 'A')
('E', 'C', 'A')
('E', 'B', 'G')
('D', 'B', 'F')
('C', 'A', 'F')
---------------


In [109]:
vc = VoicingCycle(c.chords(3), (1,5, 3)) # cycle: 1->5->3->1...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle3, voicings) # close voicing, spread voicing

('C', 'E', 'G')
('B', 'E', 'G')
('B', 'D', 'G')
('B', 'D', 'F')
('A', 'D', 'F')
('A', 'C', 'F')
('A', 'C', 'E')
---------------
('G', 'C', 'E')
('G', 'B', 'E')
('G', 'B', 'D')
('F', 'B', 'D')
('F', 'A', 'D')
('F', 'A', 'C')
('E', 'A', 'C')
---------------
('E', 'G', 'C')
('E', 'G', 'B')
('D', 'G', 'B')
('D', 'F', 'B')
('D', 'F', 'A')
('C', 'F', 'A')
('C', 'E', 'A')
---------------
('C', 'G', 'E')
('B', 'G', 'E')
('B', 'G', 'D')
('B', 'F', 'D')
('A', 'F', 'D')
('A', 'F', 'C')
('A', 'E', 'C')
---------------
('G', 'E', 'C')
('G', 'E', 'B')
('G', 'D', 'B')
('F', 'D', 'B')
('F', 'D', 'A')
('F', 'C', 'A')
('E', 'C', 'A')
---------------
('E', 'C', 'G')
('E', 'B', 'G')
('D', 'B', 'G')
('D', 'B', 'F')
('D', 'A', 'F')
('C', 'A', 'F')
('C', 'A', 'E')
---------------


In [110]:
# start with C Melodic Minor Triads
c = Diatonic("C", Scale.M_MINOR)

In [111]:
vc = VoicingCycle(c.chords(3), (5,3,1)) # cycle: 5->3->1->5...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'Eb', 'G')
('A', 'D', 'F')
('G', 'B', 'Eb')
('F', 'A', 'C')
('D', 'G', 'B')
('C', 'Eb', 'A')
('B', 'D', 'F')
---------------
('G', 'C', 'Eb')
('F', 'A', 'D')
('Eb', 'G', 'B')
('C', 'F', 'A')
('B', 'D', 'G')
('A', 'C', 'Eb')
('F', 'B', 'D')
---------------
('Eb', 'G', 'C')
('D', 'F', 'A')
('B', 'Eb', 'G')
('A', 'C', 'F')
('G', 'B', 'D')
('Eb', 'A', 'C')
('D', 'F', 'B')
---------------
('C', 'G', 'Eb')
('A', 'F', 'D')
('G', 'Eb', 'B')
('F', 'C', 'A')
('D', 'B', 'G')
('C', 'A', 'Eb')
('B', 'F', 'D')
---------------
('G', 'Eb', 'C')
('F', 'D', 'A')
('Eb', 'B', 'G')
('C', 'A', 'F')
('B', 'G', 'D')
('A', 'Eb', 'C')
('F', 'D', 'B')
---------------
('Eb', 'C', 'G')
('D', 'A', 'F')
('B', 'G', 'Eb')
('A', 'F', 'C')
('G', 'D', 'B')
('Eb', 'C', 'A')
('D', 'B', 'F')
---------------


In [112]:
# start with C Harmonic Minor Triads
c = Diatonic("C", Scale.H_MINOR)

In [113]:
vc = VoicingCycle(c.chords(3), (5,3,1)) # cycle: 5->3->1->5...
voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'Eb', 'G')
('Ab', 'D', 'F')
('G', 'B', 'Eb')
('F', 'Ab', 'C')
('D', 'G', 'B')
('C', 'Eb', 'Ab')
('B', 'D', 'F')
---------------
('G', 'C', 'Eb')
('F', 'Ab', 'D')
('Eb', 'G', 'B')
('C', 'F', 'Ab')
('B', 'D', 'G')
('Ab', 'C', 'Eb')
('F', 'B', 'D')
---------------
('Eb', 'G', 'C')
('D', 'F', 'Ab')
('B', 'Eb', 'G')
('Ab', 'C', 'F')
('G', 'B', 'D')
('Eb', 'Ab', 'C')
('D', 'F', 'B')
---------------
('C', 'G', 'Eb')
('Ab', 'F', 'D')
('G', 'Eb', 'B')
('F', 'C', 'Ab')
('D', 'B', 'G')
('C', 'Ab', 'Eb')
('B', 'F', 'D')
---------------
('G', 'Eb', 'C')
('F', 'D', 'Ab')
('Eb', 'B', 'G')
('C', 'Ab', 'F')
('B', 'G', 'D')
('Ab', 'Eb', 'C')
('F', 'D', 'B')
---------------
('Eb', 'C', 'G')
('D', 'Ab', 'F')
('B', 'G', 'Eb')
('Ab', 'F', 'C')
('G', 'D', 'B')
('Eb', 'C', 'Ab')
('D', 'B', 'F')
---------------


In [114]:
# start with C Major 7sth Chords
c = Diatonic("C", Scale.MAJOR)

In [115]:
vc = VoicingCycle(c.chords(4), (5,3,1,7)) # cycle: 5->3->1->7->5...
voicings = [(1,3,5,7)] # close voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'E', 'G', 'B')
('C', 'D', 'F', 'A')
('B', 'D', 'E', 'G')
('A', 'C', 'E', 'F')
('G', 'B', 'D', 'F')
('G', 'A', 'C', 'E')
('F', 'A', 'B', 'D')
---------------
('E', 'G', 'B', 'C')
('D', 'F', 'A', 'C')
('D', 'E', 'G', 'B')
('C', 'E', 'F', 'A')
('B', 'D', 'F', 'G')
('A', 'C', 'E', 'G')
('A', 'B', 'D', 'F')
---------------
('G', 'B', 'C', 'E')
('F', 'A', 'C', 'D')
('E', 'G', 'B', 'D')
('E', 'F', 'A', 'C')
('D', 'F', 'G', 'B')
('C', 'E', 'G', 'A')
('B', 'D', 'F', 'A')
---------------
('B', 'C', 'E', 'G')
('A', 'C', 'D', 'F')
('G', 'B', 'D', 'E')
('F', 'A', 'C', 'E')
('F', 'G', 'B', 'D')
('E', 'G', 'A', 'C')
('D', 'F', 'A', 'B')
---------------


In [116]:
voicings = [(1,5,7,3)] # drop-2 voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'G', 'B', 'E')
('C', 'F', 'A', 'D')
('B', 'E', 'G', 'D')
('A', 'E', 'F', 'C')
('G', 'D', 'F', 'B')
('G', 'C', 'E', 'A')
('F', 'B', 'D', 'A')
---------------
('E', 'B', 'C', 'G')
('D', 'A', 'C', 'F')
('D', 'G', 'B', 'E')
('C', 'F', 'A', 'E')
('B', 'F', 'G', 'D')
('A', 'E', 'G', 'C')
('A', 'D', 'F', 'B')
---------------
('G', 'C', 'E', 'B')
('F', 'C', 'D', 'A')
('E', 'B', 'D', 'G')
('E', 'A', 'C', 'F')
('D', 'G', 'B', 'F')
('C', 'G', 'A', 'E')
('B', 'F', 'A', 'D')
---------------
('B', 'E', 'G', 'C')
('A', 'D', 'F', 'C')
('G', 'D', 'E', 'B')
('F', 'C', 'E', 'A')
('F', 'B', 'D', 'G')
('E', 'A', 'C', 'G')
('D', 'A', 'B', 'F')
---------------


In [117]:
voicings = [(1,7,3,5)] # drop 3 voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'B', 'E', 'G')
('C', 'A', 'D', 'F')
('B', 'G', 'D', 'E')
('A', 'F', 'C', 'E')
('G', 'F', 'B', 'D')
('G', 'E', 'A', 'C')
('F', 'D', 'A', 'B')
---------------
('E', 'C', 'G', 'B')
('D', 'C', 'F', 'A')
('D', 'B', 'E', 'G')
('C', 'A', 'E', 'F')
('B', 'G', 'D', 'F')
('A', 'G', 'C', 'E')
('A', 'F', 'B', 'D')
---------------
('G', 'E', 'B', 'C')
('F', 'D', 'A', 'C')
('E', 'D', 'G', 'B')
('E', 'C', 'F', 'A')
('D', 'B', 'F', 'G')
('C', 'A', 'E', 'G')
('B', 'A', 'D', 'F')
---------------
('B', 'G', 'C', 'E')
('A', 'F', 'C', 'D')
('G', 'E', 'B', 'D')
('F', 'E', 'A', 'C')
('F', 'D', 'G', 'B')
('E', 'C', 'G', 'A')
('D', 'B', 'F', 'A')
---------------


In [118]:
voicings = [(1,3,7,5)] # drop 2 & 3 voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'E', 'B', 'G')
('C', 'D', 'A', 'F')
('B', 'D', 'G', 'E')
('A', 'C', 'F', 'E')
('G', 'B', 'F', 'D')
('G', 'A', 'E', 'C')
('F', 'A', 'D', 'B')
---------------
('E', 'G', 'C', 'B')
('D', 'F', 'C', 'A')
('D', 'E', 'B', 'G')
('C', 'E', 'A', 'F')
('B', 'D', 'G', 'F')
('A', 'C', 'G', 'E')
('A', 'B', 'F', 'D')
---------------
('G', 'B', 'E', 'C')
('F', 'A', 'D', 'C')
('E', 'G', 'D', 'B')
('E', 'F', 'C', 'A')
('D', 'F', 'B', 'G')
('C', 'E', 'A', 'G')
('B', 'D', 'A', 'F')
---------------
('B', 'C', 'G', 'E')
('A', 'C', 'F', 'D')
('G', 'B', 'E', 'D')
('F', 'A', 'E', 'C')
('F', 'G', 'D', 'B')
('E', 'G', 'C', 'A')
('D', 'F', 'B', 'A')
---------------


In [119]:
voicings = [(1,5,3,7)] # drop 2 & 4 voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'G', 'E', 'B')
('C', 'F', 'D', 'A')
('B', 'E', 'D', 'G')
('A', 'E', 'C', 'F')
('G', 'D', 'B', 'F')
('G', 'C', 'A', 'E')
('F', 'B', 'A', 'D')
---------------
('E', 'B', 'G', 'C')
('D', 'A', 'F', 'C')
('D', 'G', 'E', 'B')
('C', 'F', 'E', 'A')
('B', 'F', 'D', 'G')
('A', 'E', 'C', 'G')
('A', 'D', 'B', 'F')
---------------
('G', 'C', 'B', 'E')
('F', 'C', 'A', 'D')
('E', 'B', 'G', 'D')
('E', 'A', 'F', 'C')
('D', 'G', 'F', 'B')
('C', 'G', 'E', 'A')
('B', 'F', 'D', 'A')
---------------
('B', 'E', 'C', 'G')
('A', 'D', 'C', 'F')
('G', 'D', 'B', 'E')
('F', 'C', 'A', 'E')
('F', 'B', 'G', 'D')
('E', 'A', 'G', 'C')
('D', 'A', 'F', 'B')
---------------


In [120]:
voicings = [(1,7,5,3)] # double drop 2 & 3 voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'B', 'G', 'E')
('C', 'A', 'F', 'D')
('B', 'G', 'E', 'D')
('A', 'F', 'E', 'C')
('G', 'F', 'D', 'B')
('G', 'E', 'C', 'A')
('F', 'D', 'B', 'A')
---------------
('E', 'C', 'B', 'G')
('D', 'C', 'A', 'F')
('D', 'B', 'G', 'E')
('C', 'A', 'F', 'E')
('B', 'G', 'F', 'D')
('A', 'G', 'E', 'C')
('A', 'F', 'D', 'B')
---------------
('G', 'E', 'C', 'B')
('F', 'D', 'C', 'A')
('E', 'D', 'B', 'G')
('E', 'C', 'A', 'F')
('D', 'B', 'G', 'F')
('C', 'A', 'G', 'E')
('B', 'A', 'F', 'D')
---------------
('B', 'G', 'E', 'C')
('A', 'F', 'D', 'C')
('G', 'E', 'D', 'B')
('F', 'E', 'C', 'A')
('F', 'D', 'B', 'G')
('E', 'C', 'A', 'G')
('D', 'B', 'A', 'F')
---------------


In [121]:
vc = VoicingCycle(c.chords(4), (5,2,1,7)) # cycle: 5->2->1->7->5...
voicings = [(1,2,5,7)] # close voicing
vc.gen(vc.Cycle2, voicings ) 

('C', 'E', 'G', 'B')
('C', 'D', 'F', 'A')
('B', 'D', 'E', 'G')
('A', 'C', 'E', 'F')
('G', 'B', 'D', 'F')
('G', 'A', 'C', 'E')
('F', 'A', 'B', 'D')
---------------
('E', 'G', 'B', 'C')
('D', 'F', 'A', 'C')
('D', 'E', 'G', 'B')
('C', 'E', 'F', 'A')
('B', 'D', 'F', 'G')
('A', 'C', 'E', 'G')
('A', 'B', 'D', 'F')
---------------
('G', 'B', 'C', 'E')
('F', 'A', 'C', 'D')
('E', 'G', 'B', 'D')
('E', 'F', 'A', 'C')
('D', 'F', 'G', 'B')
('C', 'E', 'G', 'A')
('B', 'D', 'F', 'A')
---------------
('B', 'C', 'E', 'G')
('A', 'C', 'D', 'F')
('G', 'B', 'D', 'E')
('F', 'A', 'C', 'E')
('F', 'G', 'B', 'D')
('E', 'G', 'A', 'C')
('D', 'F', 'A', 'B')
---------------


Create an inline user-interface for generating any combination in the book (in all keys and cycles)

In [131]:
from tables import description
from google.colab import widgets
import ipywidgets
from IPython.display import display, clear_output

bw = ipywidgets.widgets.Button(description="Generate", colors=['Green'])
# create output widget
output = ipywidgets.widgets.Output()

def on_button_clicked(b):
  with output:
    clear_output()  
    scale = {'Major': Scale.MAJOR, 'Mel. Minor': Scale.M_MINOR,'Har. Minor': Scale.H_MINOR }

    dd = Diatonic(kw.value, scale[sw.value])
    if cw.value == 'Seventh':
      vc = VoicingCycle(dd.chords(4), (5,3,1,7)) # cycle: 5->3->1->5...
      voicings = [(1,3,5,7)] # close voicing
      vc.gen(cycle, voicings ) 
    else:
      vc = VoicingCycle(dd.chords(3), (5,3,1)) # cycle: 5->3->1->5...
      voicings = [(1,3,5), (1,5,3)] # close voicing, spread voicing
    
    cycle = {'Cycle2':vc.Cycle2, 'Cycle3':vc.Cycle3, 'Cycle4':vc.Cycle4, 'Cycle5':vc.Cycle5, 'Cycle6':vc.Cycle6, 'Cycle7':vc.Cycle7}
    vc.gen(cycle[lw.value], voicings ) 
  display(output)

bw.on_click(on_button_clicked)
bw.style.button_color = 'lightgreen'

kw = ipywidgets.widgets.Dropdown(
    options=Diatonic._notes_flat,
    value='C',
    size=10,
    description='Key:'
)

sw = ipywidgets.widgets.Dropdown(
    options=['Major', 'Mel. Minor', 'Har. Minor'],
    value='Major',
    description='Scale:'
)

cw = ipywidgets.widgets.Dropdown(
    options=['Triad', 'Seventh'],
    value='Triad',
    description='Chord:'
)

lw = ipywidgets.widgets.Dropdown(
    options =  ['Cycle2', 'Cycle3', 'Cycle4', 'Cycle5', 'Cycle6', 'Cycle7'],
    value='Cycle2',
    description='Cycle:'
)

ipywidgets.widgets.HBox([bw, kw, sw, cw, lw])


Output()

Output()

Output()